### Perspective transformation using manually selected Region of Interest(ROI)

#### ROI Selector Module

* For this, you would need to use `four_point_roi_selector.py` module from this repository.

#### select ROI
Get ROI points. These are used to perform the perspective transform.

* Note: If you re-run this code for the second time, you won't be able to see the ROI drawn on the image. You need to reset it (by pressing r). This only happens in jupyter notebook because the points remain in the session.

* Instructions appear when you call the ROI selector. See below.

#### Image warping
* Remember: This depends on what order you have selected the ROI points. Select carefully.
* Desired order: tr, tl, bl, br to enclose a 4-point figure.

In [15]:
import numpy as np
import cv2
import sys, os
from four_point_roi_selector import ROIDefiner

In [16]:
def select_roi(frame):
    """
    select roi in order 
    top-left, top-right, bottom-right, bottom-left
    """
    copy_frame = frame.copy()
    roi_definer = ROIDefiner()
    boundary_points = roi_definer.define_roi(frame, copy_frame)
    print("selected coordinates: ", boundary_points)
    return np.array(boundary_points, dtype=np.float32)

In [18]:
import numpy as np
import cv2

def four_point_transform(image, pts):
    if len(pts) != 4:
        raise ValueError("Input 'pts' must contain exactly four points.")

    rect = np.array(pts, dtype="float32")

    try:
        # compute the width and height of the new image
        widthA = np.linalg.norm(rect[1] - rect[0])
        widthB = np.linalg.norm(rect[2] - rect[3])
        heightA = np.linalg.norm(rect[2] - rect[1])
        heightB = np.linalg.norm(rect[3] - rect[0])

        maxWidth = max(int(widthA), int(widthB))
        maxHeight = max(int(heightA), int(heightB))

        dst = np.array([
            [0, 0],
            [maxWidth - 1, 0],
            [maxWidth - 1, maxHeight - 1],
            [0, maxHeight - 1]], dtype="float32")

        # compute the perspective transform matrix
        M = cv2.getPerspectiveTransform(rect, dst)

        # apply the perspective transform
        warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

        return warped

    except Exception as e:
        raise Exception("An error occurred during perspective transformation: {}".format(str(e)))


In [19]:
if __name__ == "__main__":
    image = cv2.imread('arco-center-ice.jpg')
    image_copy = image.copy()
    # resize, if needed
    # image = cv2.resize(image, None, fx = 0.5, fy = 0.5, interpolation = cv2.INTER_CUBIC)
    # image = cv2.resize(image, (640, 480), interpolation = cv2.INTER_CUBIC)
    pts = select_roi(image_copy)

    warped_image = four_point_transform(image, pts)

    cv2.imshow('Warped Image', warped_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


Image resolution:  (1037, 1560, 3)
--------------------------------------------------------------
Define roi, select Left->Top->Right->Bottom points on an image
Please follow the below order while selecting roi
--------------------------------------------------------------
Select --> lower_left_x, lower_left_y  point
Select --> upper_left_x, upper_left_y  point
Select --> upper_right_x, upper_right_y  point
Select --> lower_right_x, lower_right_y  point
---------------------------------------------------------------------
---------------------------------------------------------------------
Press: 
'c': Confirm
'r': Reset
'q': Quit
---------------------------------------------------------------------

selected coordinates:  [(613, 312), (959, 316), (1502, 883), (39, 887)]
